In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=S6TlvQ_-FeYswTvOaJVqZaWn1F4HBZebLj-xZmIEvQ8&tc=iyv4_ksjVk6FBrS9J7J0QZ-mI_TR3diXvTSLx948fH4&cc=vW4rIPT2lyiRTsJ44iBS8ahN2SsD5bvtj03W9Zrs30A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuFbTYMxF8UPLYNzJJB_flPr8ByJ9y0V74b_WW0Rr_uC-4HwJJHOkI

Successfully saved authorization token.


In [ ]:
#parameters
PARAMS = { 'START_DATE': '2019-11-15',
            'STOP_DATE': '2020-04-30',        
            'CLOUD_FILTER': 100,
            'CLD_PRB_THRESH' : 30,
            'NIR_DRK_THRESH': 0.15,
            'CLD_PRJ_DIST':1,
            'BUFFER':50,
            'TABLE':ee.FeatureCollection('users/dgaso/UY_19-20_1ra/Field_50'),
            'FIELD': 'Field025',
            'PROJECTION': 'EPSG:32615',
            'INTERVAL':15,
            'EXPORT_FOLDER': 'S2_Features_US',
            'SCALE':20
            }

#area of interest
aoi = PARAMS['TABLE'].geometry();
#day intervals 
biweeks = ee.List.sequence(320,350,PARAMS['INTERVAL']); 
biweeks2 = ee.List.sequence(0,105,PARAMS['INTERVAL']); 
#list of indices to download
index = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12', 'NDMI', 'NDVI', 'GNDVI', 'MCARI', 'TCARI', 'CIG', 'WDRVI']

HELPER FUNCTIONS

In [ ]:
#Google cloud mask functions
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', PARAMS['CLOUD_FILTER'])))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(PARAMS['CLD_PRB_THRESH']).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(PARAMS['NIR_DRK_THRESH']*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, PARAMS['CLD_PRJ_DIST']*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(PARAMS['BUFFER']*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

#add vegetation indices
def addIndices(img):
    ndmi = img.normalizedDifference(['B8','B11']).rename('NDMI')
    ndvi = img.normalizedDifference(['B8','B4']).rename('NDVI')
    gndvi = img.normalizedDifference(['B8','B3']).rename('GNDVI')
    mcari = img.expression( \
      ' ((X - Y) - 0.2*(X-Z)) * (X/Y)', { \
        'X': img.select('B5'), \
        'Y': img.select('B4'),  \
        'Z': img.select('B3')}).rename('MCARI')
    tcari = img.expression( \
      '3* ((X - Y) - 0.2*(X-Z) * (X/Y))', { \
        'X': img.select('B5'), \
        'Y': img.select('B4'),  \
        'Z': img.select('B3')}).rename('TCARI')
    cig = img.expression( \
      ' (X / Y) - 1', { \
        'X': img.select('B8'), \
        'Y': img.select('B3')}).rename('CIG')
    wdrvi = img.expression( \
      ' (0.5* X - Y) / (0.5* X + Y)', { \
        'X': img.select('B8'), \
        'Y': img.select('B4')}).rename('WDRVI')

    return img.addBands(ndmi).addBands(ndvi).addBands(gndvi).addBands(mcari).addBands(tcari).addBands(cig).addBands(wdrvi)

#temporal moving window average
def mov_ave(m):
  m = ee.Number(m)
  return withVI.filter(ee.Filter.calendarRange(m, m.add(15), 'day_of_year')) \
          .filterBounds(aoi) \
           .mean().set('day_of_year', m)

In [ ]:
#Sentinel-2 collection
s2_sr_cld_col = get_s2_sr_cld_col(aoi, PARAMS['START_DATE'], PARAMS['STOP_DATE'])
#cloud masked collection
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask) \
                             .map(apply_cld_shdw_mask))
#convert to float
S2 = s2_sr_median.map(lambda image: image.toFloat())
#add indices
withVI = S2.map(addIndices)

In [ ]:
#temporal moving average
Biweekly_mean =  ee.ImageCollection.fromImages(biweeks.map(mov_ave));
Biweekly_mean2 =  ee.ImageCollection.fromImages(biweeks2.map(mov_ave));
#merge collections
Biweekly_mean = Biweekly_mean.merge(Biweekly_mean2).sort('system:index')
#clip to field boundary
clip_aoi = Biweekly_mean.map(lambda image: image.clip(aoi))

In [ ]:
#export images per feature
for i in range(0, len(index)):

  download_aoi = clip_aoi.map(lambda image: image.select(index[i])).toBands()
  task = ee.batch.Export.image.toDrive( 
              image= download_aoi, 
              description= index[i] + '_' + PARAMS['FIELD'], 
              fileNamePrefix= index[i] + '_' + PARAMS['FIELD'],
              scale= PARAMS['SCALE'],
              crs= PARAMS['PROJECTION'],
              fileFormat= 'GeoTIFF', 
              folder= PARAMS['EXPORT_FOLDER'], 
              region= aoi, 
              maxPixels= 1e13)

  task.start() 
